<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg"  width=300, align="right">
<br>
<br>
<br>
<br>
<br>

# **Expert Systems, Rule‑Based Programming `PyKnown`**
---


In [ ]:
#@title **Identificação do Grupo**

#@markdown Integrantes do Grupo, nome completo em orgem alfabética
Aluno1 = '10403109, Erik Samuel Viana Hsu' #@param {type:"string"}
Aluno2 = '10400995, Mateus Kenzo Iochimoto' #@param {type:"string"}
Aluno3 = '10400764, Thiago Shihan Cardoso Toma' #@param {type:"string"}



# **Resumo**

Neste trabalho, foi proposto que desenvolvêssemos um conjunto original de regras (no mínimo 16) para classificar objetos ou temas de nossa escolha. Optamos por criar um sistema baseado em regras para a classificação de aves/pássaros, utilizando a taxonomia de Sibley–Ahlquist como referência.

Nosso sistema de inferência opera a partir da identificação de características das aves (como formato do bico, tipo de voo, alimentação, habitat e hábitos diurnos/noturnos) e, com base nesses fatos, determina a qual ordem pertencem. Por exemplo, aves carnívoras, com bico em forma de gancho e hábitos noturnos são classificadas como Strigiformes (corujas), enquanto aquelas com bico e alimentação semelhantes porém hábitos diurnos são associadas aos Falconiformes (águias e gaviões).

A implementação foi feita na linguagem Python utilizando a biblioteca pyknow, que permite a criação de sistemas especialistas baseados em regras. O sistema foi testado com diferentes combinações de características para validar sua precisão na identificação de ordens de aves. As referências da nossa pesquisa estão no último tópico.

# **Implementação**



Desinstalamos yfinance por ser incompativel com pyknow.


In [ ]:
!pip uninstall yfinance

Instalamos biblioteca pyknow para criar o sistema de inferências.

In [ ]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-2hijjo2p
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-2hijjo2p
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done


Aqui importamos a biblioteca pyknow e definimos a classe "Passaro" que representa os fatos no sistema especialista, cada instância de Passaro vai armazenar características como bico, tamanho, habitat e etc.


In [ ]:
import collections.abc
collections.Mapping = collections.abc.Mapping
from pyknow import *

class Passaro(Fact):
    """Fatos que representam as características de um pássaro"""
    pass

Aqui definimos a nossa engine que irá processar as regras do sistema e classificar a nossa instância de passaro de acordo com as regras definidas e os fatos fornecidos.

Ela é composta pelas regras do nosso sistema de classificação, para isso, tomamos como base a "Taxonomia de aves de Sibley-Ahlquist", uma classificação de aves proposta por Charles Sibley e Jon E. Ahlquis no final da década de 1970 e ao longo da década de 1980.

Classificamos cada ordem de ave baseada em critérios como tamanho, habitat, alimentação, formato do bico e etc...

In [ ]:
class ClassificadorAves(KnowledgeEngine):

    @Rule(OR(
        AND(Passaro(tamanho="muito grande"), Passaro(voo="não voa"), Passaro(pes_poderosos="sim")),
        Passaro(tamanho="grande", voo="não voa", pes_poderosos="sim"),
    ))
    def struthioniformes(self):
        self.declare(Fact(ordem='Struthioniformes', nome='Avestruz, ema'))

    @Rule(OR(
        AND(Passaro(tamanho="médio"), Passaro(voo="curto"), Passaro(habitat="terrestre"), Passaro(pernas_fortes="sim")),
        Passaro(tamanho="médio", voo="curto", habitat="floresta", pernas_fortes="sim")
    ))
    def galliformes(self):
        self.declare(Fact(ordem='Galliformes', nome='Faisões, Perus, Mutuns, Jacus'))

    @Rule(OR(
        AND(Passaro(tamanho="médio"),Passaro(voo = "médio"), Passaro(habitat="aquático"), Passaro(bico="largo")),
        Passaro(tamanho="médio", voo="longo", habitat="aquático", bico="largo")
    ))
    def anseriformes(self):
        self.declare(Fact(ordem='Anseriformes', nome='Patos, Gansos, Cisnes'))

    @Rule(OR(
        AND(Passaro(tamanho="pequeno"), Passaro(canto_melodico="sim"), Passaro(habitat="urbano")),
        Passaro(tamanho="pequeno", canto_melodico="sim", habitat="campo")
    ))
    def passeriformes(self):
        self.declare(Fact(ordem='Passeriformes', nome='Pássaros como Pardal e Bem-te-vi'))

    @Rule(OR(
        AND(Passaro(tamanho="médio"), Passaro(bico="grande"), Passaro(habitat="floresta"), Passaro(alimentacao="frutas")),
        Passaro(tamanho="médio", bico="grande", habitat="floresta", alimentacao="insetos")
    ))
    def piciformes(self):
        self.declare(Fact(ordem='Piciformes', nome='Pica-paus, Tucanos, Ararambas'))

    @Rule(OR(
        AND(Passaro(bico_formato="gancho"), Passaro(alimentação="carne")),
        AND(Passaro(bico_formato="gancho"), Passaro(voo="rápido"), Passaro(alimentação="carne"))
    ))
    def aves_rapina(self):
        self.declare(Fact(aves_rapina=True))

    @Rule(AND(Fact(aves_rapina=True), Passaro(atividade="noturno")))
    def strigiformes(self):
        self.declare(Fact(ordem='Strigiformes', nome='Coruja'))

    @Rule(AND(Fact(aves_rapina=True), Passaro(atividade="diurno")))
    def falconiformes(self):
        self.declare(Fact(ordem='Falconiformes', nome='Águia, Gavião, Carcará'))

    @Rule(Passaro(habitat="costeiro", bico="longo", pernas_longas="sim"))
    def charadriiformes(self):
        self.declare(Fact(ordem='Charadriiformes', nome='Gaivotas, Quero-quero, Trinta-réis'))

    @Rule(Passaro(voo="curto", ninho="solo", alimentação="sementes"))
    def tinamiformes(self):
        self.declare(Fact(ordem='Tinamiformes', nome='Inhambus, Macucos, Codornas'))

    @Rule(Passaro(habitat="marinho"), Passaro(bico="grande"), Passaro(pes_palmados="sim"))
    def pelecaniformes(self):
        self.declare(Fact(ordem='Pelecaniformes', nome='Pelicanos, Biguás, Fragatas'))

    @Rule(OR(
    AND(Passaro(habitat="polar"), Passaro(voo="não voa"), Passaro(pes_palmados="sim")),
    Passaro(habitat="polar", voo="não voa", pes_palmados="sim")))
    def sphenisciformes(self):
        self.declare(Fact(ordem='Sphenisciformes', nome='Pinguins'))

    @Rule(Passaro(habitat="floresta", comportamento_parasitismo="sim"))
    def cuculiformes(self):
        self.declare(Fact(ordem='Cuculiformes', nome='Cucos, Cigana'))

    @Rule(OR(
        AND(Passaro(habitat="floresta"), Passaro(cores_vivas="sim"), Passaro(bico="curto"), Passaro(alimentacao="insetos")),
        Passaro(habitat="floresta", cores_vivas="sim", bico="curto", alimentacao="frutas")
    ))
    def trogoniformes(self):
        self.declare(Fact(ordem='Trogoniformes', nome='Surucuás'))

    @Rule(OR(
        AND(Passaro(tamanho="muito pequeno"), Passaro(voo="rápido"), Passaro(bico="fino"), Passaro(alimentacao="néctar")),
        Passaro(tamanho="muito pequeno", habitat="floresta", bico="fino", alimentacao="néctar", voo = "pairado")
    ))
    def apodiformes(self):
        self.declare(Fact(ordem='Apodiformes', nome='Beija-flores, Andorinhões'))

    @Rule(Fact(ordem=MATCH.o, nome=MATCH.n))
    def print_resultado(self, o, n):
        print(f"Ordem: {o} - {n}")

    def factz(self, l):
      for x in l:
          self.declare(x)

# **Resultados (Testes)**

### **Teste 1**

In [ ]:
teste1 = ClassificadorAves()
teste1.reset()
teste1.factz([
    Passaro(bico_formato="gancho"),
    Passaro(alimentação="carne"),
    Passaro(atividade="noturno")
])
teste1.run()


Ordem: Strigiformes - Coruja


Para nosso primeiro teste declaramos que o passaro tem o bico no formato "gancho" e alimentação carnívora. De acordo com as regras definidas, esses fatos ja mostram que o passaro é uma ave de rápina.

Regra específica ilustrada a seguir:

In [ ]:
 @Rule(OR(
        AND(Passaro(bico_formato="gancho"), Passaro(alimentação="carne")),
        AND(Passaro(bico_formato="gancho"), Passaro(voo="rápido"), Passaro(alimentação="carne"))
    ))
    def aves_rapina(self):
        self.declare(Fact(aves_rapina=True))

A regra adicional:

In [ ]:
 @Rule(AND(Fact(aves_rapina=True), Passaro(atividade="noturno")))
    def strigiformes(self):
        self.declare(Fact(ordem='Strigiformes', nome='Coruja'))

distingue os Stringiformes (noturnos) dos Falconiformes(diurnos), como no nosso teste também classificamos o passaro como noturno, nos restou apenas a classe Stringiforme que corresponde as características (fatos) definidas para esse caso de teste.

In [ ]:
Passaro(bico_formato="gancho"),
Passaro(alimentação="carne"),
Passaro(atividade="noturno")

### **Teste 2**


In [ ]:
teste2 = ClassificadorAves()
teste2.reset()
teste2.factz([
    Passaro(tamanho="muito pequeno"),
    Passaro(voo="rápido"),
    Passaro(bico="fino"),
    Passaro(alimentacao="néctar")
])
teste2.run()

Ordem: Apodiformes - Beija-flores, Andorinhões


Para esse segundo teste definimos as características muito pequeno, voo rápido, bico fino e alientação de néctar.

Essas características correspondem apenas a Ordem dos Apodiformes (beija-flores), como ilustrado na regra a seguir:

In [ ]:
@Rule(OR(
        AND(Passaro(tamanho="muito pequeno"), Passaro(voo="rápido"), Passaro(bico="fino"), Passaro(alimentacao="néctar")),
        Passaro(tamanho="muito pequeno", voo="rápido", bico="fino", alimentacao="néctar")
    ))
    def apodiformes(self):
        self.declare(Fact(ordem='Apodiformes', nome='Beija-flores, Andorinhões'))

### **Teste 3**


In [ ]:
    teste_3 = ClassificadorAves()
    teste_3.reset()
    teste_3.factz([
        Passaro(habitat="polar"),
        Passaro(voo="não voa"),
        Passaro(pes_palmados="sim")
    ])
    teste_3.run()

Ordem: Sphenisciformes - Pinguins


Último teste, dessa vez com as característcias da ordem Sphenisciformes (Pinguins).

# **Referências**

1.	BIRDING. Strigiformes. Disponível em: https://www.birding.in/orders/strigiformes.htm. Acesso em: 25 mar. 2025.

2.	INFOESCOLA. Aves. Disponível em: https://www.infoescola.com/biologia/aves/. Acesso em: 25 mar. 2025.

3.	WIKIPEDIA. Sibley–Ahlquist taxonomy of birds. Disponível em: https://en.wikipedia.org/wiki/Sibley%E2%80%93Ahlquist_taxonomy_of_birds. Acesso em: 25 mar. 2025.

---

In [ ]:
#@title **Avaliação**
Resumo = 10 #@param {type:"slider", min:0, max:10, step:1}

Implementacao = 10 #@param {type:"slider", min:0, max:10, step:1}

Resultados = 10 #@param {type:"slider", min:0, max:10, step:1}

Referencias = 10 #@param {type:"slider", min:0, max:10, step:1}

Geral = 10 #@param {type:"slider", min:0, max:10, step:1}








In [ ]:
#@title **Nota Final**
nota = Resumo + Implementacao + Resultados + Referencias + Geral

nota = nota / 5

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_tia = []
lista_nome = []

for i in range(1,6):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_tia.append(lista[0]); lista_nome.append(lista[1].upper())")

alunos['tia'] = lista_tia
alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
display(alunos)

Nota final do trabalho 7.8


NameError: name 'Aluno4' is not defined